##Hei! Her kan du se en rekke eksempler på bruk av tabellen `nvdb_vegpunkter` i praksis.<br>Merk at vegpunkter tabellen alltid kun holder inneværende mnd,<br> og historikken ligger i "_history".<br><br>NB! I noen tilfeller har vi brukt _history varianten av tabellen, i de eksemplene der vi viser en spesifikk punkt_id. Dette for at eksempelet ikke skal slutte å virke en gang i fremtiden.

In [ ]:
project = 'saga-data'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

Authenticated


In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

## Eksempel 1

Jeg har noen veibredde-data som jeg gjerne skulle ha beriket med fartsgrense.

In [ ]:
query = f"""
-- Vi lager en rad for å simulere dataene.
with veibredde as (
  select 72611 as veglenkesekvensid, 0.578742 as startposisjon, 1.0 as sluttposisjon, 6 as veibredde
)

-- Vi må koble dette til vegpunktene for å enkelt kunne koble på fartsgrenser.
-- Først henter vi vegpunktene
, vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
)

-- Og kobler veibreddedataene til vegpunktene for å få informasjonen på metersnivå
, joined_step_1 as (
  select
    a.punkt_id
  , a.veglenkesekvensid
  , a.relativ_posisjon
  , b.veibredde
  from vegpunkter as a
  inner join veibredde as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and a.relativ_posisjon between b.startposisjon and b.sluttposisjon
)

-- Vi henter så fartsgrenser
, fartsgrenser as (
  select
    egenskaper.fartsgrenseverdi as fartsgrense
  , ls.veglenkesekvensid
  , ls.startposisjon
  , ls.sluttposisjon
  from saga-nvdb-prod-vlmh.standardized.vegobjekter_fartsgrense,
  unnest(lokasjon.stedfestinger) as ls
  where (metadata.sluttdato is null or metadata.sluttdato >= current_date())
)

-- Vi kobler fartsgrenser på vegpunktene
, joined_all as (
  select
    a.*
  , b.fartsgrense
  from joined_step_1 as a
  inner join fartsgrenser as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and a.relativ_posisjon between b.startposisjon and b.sluttposisjon
)

-- Og ruller det opp igjen til det originale formatet
  select
    veglenkesekvensid
  , veibredde
  , fartsgrense
  , min(relativ_posisjon) as startposisjon
  , max(relativ_posisjon) as sluttposisjon
  from joined_all
  group by 1,2,3
  ;
"""

client.query(query).to_dataframe()

,veglenkesekvensid,veibredde,fartsgrense,startposisjon,sluttposisjon
0,72611,6,80,0.578742,1.0


## Eksempel 2
Jeg har fått noen data med såkalt "lokasjon.stedfestinger.kortform".
For eksempel "0.01587524-0.2725013@971774". Hvor i Norge er dette?

In [ ]:
query = f"""
-- Vi lager en rad for å simulere dataene.
with eksempeldata as (
  select "0.01587524-0.2725013@971774" as kortform, 42 as measurement
)

-- Dette er egentlig veglenkesekvens id og relative posisjoner, så dataene må brytes opp litt først.
, eksempeldata_brutt_opp_steg1 as (
  select
    split(kortform, '@')[OFFSET(0)] as start_til_sluttposisjon
  , split(kortform, '@')[OFFSET(1)] as veglenkesekvensid
  from eksempeldata
)

-- Deretter kaller vi variablene det som dataene representerer
, eksempeldata_brutt_opp_steg2 as (
  select
    cast(veglenkesekvensid as int64) as veglenkesekvensid
  , cast(SPLIT(start_til_sluttposisjon, '-')[OFFSET(0)] as float64) as startposisjon
  , cast(SPLIT(start_til_sluttposisjon, '-')[OFFSET(1)] as float64) as sluttposisjon
  from eksempeldata_brutt_opp_steg1
)

-- Henter inn vegpunktene
, vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
)

-- Og kobler det hele sammen
, joined_to_vegpunkter as (
  select
    vegkategori
  , vegnummer
  , kommunenummer
  , sum(avstand_til_neste_punkt) as ca_antall_meter
  from eksempeldata_brutt_opp_steg2 as a
  inner join vegpunkter as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and b.relativ_posisjon between a.startposisjon and a.sluttposisjon
  group by 1,2,3
  order by 1,2,3
)

-- Så lurer vi på hvilken kommune som kommunenummeret tilhører - vi henter det inn
, join_kommunenavn as (
  select
    a.*, b.kommunenavn
  from joined_to_vegpunkter as a
  left join saga-data.geografi.kommuner_2020_v2 as b
    on a.kommunenummer = b.kommunenummer
)

select * from join_kommunenavn
"""

client.query(query).to_dataframe()

,vegkategori,vegnummer,kommunenummer,ca_antall_meter,kommunenavn
0,E,6,3001,1662.0,Halden


Svar: Stedfestingen er på E6 i Halden, og er ca 1.66 km lang.

## Eksempel 3
Jeg har to datasett som kun er geografiske objekter (linestrings) men de har ikke noe mer info. Hvordan kan jeg koble datasettene sammen der geografien overlapper, og berike dataene med informasjon om vegnettet fra NVDB Vegpunkter)?

In [ ]:
query = f"""
-- Vi lager først begge eksempeldatasettene
with data1 as (
  select
    172 as id
  , ST_GeogFromText('LINESTRING(10.227466 59.723153, 10.22756 59.723072, 10.22765 59.722985\
  , 10.227688 59.722949, 10.227723 59.722912, 10.227759 59.722874, 10.227794 59.722838, 10.227828 59.7228\
  , 10.227862 59.722763, 10.227896 59.722725, 10.227928 59.722688, 10.22796 59.722649, 10.22799 59.722612)') as geometri
  , 42 as measurement_one
)

, data2 as (
  select
    172 as id
  , ST_GeogFromText('LINESTRING(10.227688 59.722949, 10.227723 59.722912, 10.227759 59.722874, 10.227794 59.722838)') as geometri
  , 78 as measurement_two
)

-- Vi beriker datasettene med kvadratiske grids for raskere påkobling mot NVDB Vegpunkter
, add_squaregrids_to_data1 as (
  select
    a.*
  , d.square_gridcell_id_small
  from
    data1 as a
  inner join saga-data.geografi.norway_square_polygons_large as b
    on st_intersects(a.geometri, b.geometri)
  inner join saga-data.geografi.norway_square_polygons_medium as c
    on b.x_index_large = c.x_index_large
    and b.y_index_large = c.y_index_large
    and st_intersects(a.geometri, c.geometri)
  inner join saga-data.geografi.norway_square_polygons_small as d
    on c.x_index_medium = d.x_index_medium
    and c.y_index_medium = d.y_index_medium
    and st_intersects(a.geometri, d.geometri)
)

, add_squaregrids_to_data2 as (
  select
    a.*
  , d.square_gridcell_id_small
  from
    data2 as a
  inner join saga-data.geografi.norway_square_polygons_large as b
    on st_intersects(a.geometri, b.geometri)
  inner join saga-data.geografi.norway_square_polygons_medium as c
    on b.x_index_large = c.x_index_large
    and b.y_index_large = c.y_index_large
    and st_intersects(a.geometri, c.geometri)
  inner join saga-data.geografi.norway_square_polygons_small as d
    on c.x_index_medium = d.x_index_medium
    and c.y_index_medium = d.y_index_medium
    and st_intersects(a.geometri, d.geometri)
)

-- Vi henter vegpunktene
, vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
)

select
  a.*, b.measurement_one, c.measurement_two
from vegpunkter as a
inner join add_squaregrids_to_data1 as b
  on a.square_gridcell_id_small = b.square_gridcell_id_small
  and st_intersects(st_buffer(b.geometri,10), a.punkt_geometri)
inner join add_squaregrids_to_data2 as c
  on a.square_gridcell_id_small = c.square_gridcell_id_small
  and st_intersects(st_buffer(c.geometri,10), a.punkt_geometri)
order by veglenkesekvensid, relativ_posisjon asc
"""

#print(query)

client.query(query).to_dataframe()

,punkt_id,punkt_geometri,punkt_geohash,grader_ost,grader_nord,veglenke_referanse,veglenkesekvensid,veglenkenummer,segmentnummer,vl_index,...,vl_har_sykkelfelt,vl_har_kollektivfelt,vl_har_bomstasjonfelt,run_time,run_yearmonth,square_gridcell_id_small,ssb_gridcell_id_1000m,h3_index_res_8,measurement_one,measurement_two
0,181259-22-7-2-6-u4x7b8pxbddkh627f7jp,POINT(10.2276094925661 59.7230241571862),u4x7b8pxbddkh627f7jp,10.227609,59.723024,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78
1,181259-22-7-2-7-u4x7b8pxbc0s9068tq0t,POINT(10.2276177413271 59.7230161833838),u4x7b8pxbc0s9068tq0t,10.227618,59.723016,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78
2,181259-22-7-2-8-u4x7b8px8zwsvyjj81hh,POINT(10.2276259900881 59.7230082095815),u4x7b8px8zwsvyjj81hh,10.227626,59.723008,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78
3,181259-22-7-2-9-u4x7b8px9nhv6sp66upe,POINT(10.2276342388491 59.7230002357792),u4x7b8px9nhv6sp66upe,10.227634,59.723000,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78
4,181259-22-7-2-10-u4x7b8px9kdvz9cxjz51,POINT(10.2276424876101 59.7229922619769),u4x7b8px9kdvz9cxjz51,10.227642,59.722992,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78
5,181259-22-7-3-0-u4x7b8px9e2d6zcb0t8e,POINT(10.22765 59.722985),u4x7b8px9e2d6zcb0t8e,10.227650,59.722985,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78
6,181259-22-7-3-1-u4x7b8px99yg4u23cz33,POINT(10.227658379991 59.7229770610612),u4x7b8px99yg4u23cz33,10.227658,59.722977,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78
7,181259-22-7-3-2-u4x7b8px9bm5f8ygg0fr,POINT(10.2276667599819 59.7229691221224),u4x7b8px9bm5f8ygg0fr,10.227667,59.722969,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78
8,181259-22-7-3-3-u4x7b8px6ngk6tmms6ej,POINT(10.2276751399729 59.7229611831836),u4x7b8px6ngk6tmms6ej,10.227675,59.722961,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78
9,181259-22-7-3-4-u4x7b8px6m3t49gzmsk5,POINT(10.2276835199638 59.7229532442448),u4x7b8px6m3t49gzmsk5,10.227684,59.722953,181259-22-7,181259,22,7,12,...,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,-1062674678070447182,22310006630000,88622ab897fffff,42,78


Vi kobler datasettene der geografien overlapper, og vi får ut masse nyttig info om punktene de treffer på vegnettet.

## Eksempel 4
Jeg vil ha geografiske linestrings for E6, omtrent mellom Trondheim og Oppdal (sør-/nordående).

In [ ]:
query = f"""
with vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
  where vegkategori = "E"
  and vegnummer = 6
  and vsr_objekt = "normal_strekning"
)

, koble_paa_kommunenavn as (
  select
    a.*, b.kommunenavn
  from vegpunkter as a
  left join saga-data.geografi.kommuner_2020_v2 as b
    on a.kommunenummer = b.kommunenummer
)

, filtrer_kommune as (
  select
    veglenkesekvensid
  , veglenke_referanse
  , vl_punktnummer
  , vl_subpunktnummer
  , punkt_geometri
  from koble_paa_kommunenavn
  -- Vi henter bare vegpunkter som ligger mellom disse to kommunene
  -- NB. Bruk av grader_nord eller grader_ost avhenger av plasseringen mellom lokasjonene du er interessert i
  -- For Oslo-Bergen ville det vært naturlig å bruke grader_ost
  where grader_nord > (select avg(grader_nord) from koble_paa_kommunenavn where kommunenavn = "Oppdal")
  and grader_nord < (select avg(grader_nord) from koble_paa_kommunenavn where kommunenavn = "Trondheim")
  order by 1,2,3,4 asc
)

select
  veglenkesekvensid
, veglenke_referanse
, st_makeline(array_agg(punkt_geometri)) as linestrings
from filtrer_kommune
group by 1,2
"""

#print(query)

client.query(query).to_dataframe()

,veglenkesekvensid,veglenke_referanse,linestrings
0,72147,72147-10-17,"LINESTRING(9.606995 62.515991, 9.6070010589517..."
1,72147,72147-11-17,"LINESTRING(9.60712 62.51616, 9.60712607181065 ..."
2,72147,72147-12-17,"LINESTRING(9.608568 62.519178, 9.6085692819606..."
3,72147,72147-13-17,"LINESTRING(9.608654 62.519842, 9.6086549979338..."
4,72147,72147-14-17,"LINESTRING(9.608897 62.521769, 9.6088992087185..."
...,...,...,...
691,3389158,3389158-5-3,"LINESTRING(10.033013 62.803493, 10.033022 62.8..."
692,3389158,3389158-6-3,"LINESTRING(10.033793 62.806612, 10.03378096100..."
693,3389158,3389158-8-3,"LINESTRING(10.034613 62.80553, 10.034615740179..."
694,3415848,3415848-2-4,"LINESTRING(10.288738 63.224913, 10.28872835355..."


Tips: Se på linestringene i GeoViz for å se om de virker OK før du bruker de.

## Eksempel 5
Hvilken kommune har flest vegpunkter med sykkelfelt?

In [ ]:
query = f"""
with vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
  where vegkategori in ("E", "R")
  and vl_type_veg = "Enkel bilveg"
)

, koble_paa_kommunenavn as (
  select
    a.*, b.kommunenavn
  from vegpunkter as a
  left join saga-data.geografi.kommuner_2020_v2 as b
    on a.kommunenummer = b.kommunenummer
)

select
  kommunenavn,
  sum(case when vl_har_sykkelfelt then 1 else 0 end) / sum(1) as andel_sykkelfelt
from koble_paa_kommunenavn
group by kommunenavn
order by andel_sykkelfelt desc
"""

#print(query)

client.query(query).to_dataframe()

,kommunenavn,andel_sykkelfelt
0,Hamar,0.069077
1,Oslo,0.044327
2,Trondheim,0.010652
3,Sandefjord,0.009282
4,Bergen,0.001690
...,...,...
223,Gol,0.000000
224,Stryn,0.000000
225,Øygarden,0.000000
226,Stjørdal,0.000000


Vi får sortert på kommuner med størst relativ andel vegpunkter for sykkelfelt øverst.

## Eksempel 6
Sjekk andelen veglenkesekvenser på Europavei fra Nordland og nordover som har en rasteplass eller dægnhvilkeplass et sted på lenkesekvensen.

In [ ]:
query = f"""
with rasteplass as (
  select
    1 as rasteplass,
    lokasjon.kommuner,
    ls.veglenkesekvensid,
    ls.startposisjon,
    ls.sluttposisjon
  from `saga-nvdb-prod-vlmh.standardized.vegobjekter_rasteplass`
  cross join unnest(lokasjon.stedfestinger) as ls
  where metadata.sluttdato is null
)

, dognhvileplass as (
  select
    1 as dognhvileplass,
    lokasjon.kommuner,
    ls.veglenkesekvensid,
    ls.startposisjon,
    ls.sluttposisjon
  from `saga-nvdb-prod-vlmh.standardized.vegobjekter_dognhvileplass`
  cross join unnest(lokasjon.stedfestinger) as ls
  where metadata.sluttdato is null
)

, vegpunkter as (
  select *
  from saga-data.nvdb.nvdb_vegpunkter
  where vegkategori = "E"
  and grader_nord > (select min(grader_nord) from saga-data.nvdb.nvdb_vegpunkter where fylkesnummer = 18)
)

, joined as (
  select
    a.*,
    coalesce(b.rasteplass, 0) as rasteplass, coalesce(c.dognhvileplass, 0) as dognhvileplass
  from vegpunkter  as a
  left join rasteplass as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and a.relativ_posisjon between b.startposisjon and b.sluttposisjon
  left join dognhvileplass as c
    on a.veglenkesekvensid = c.veglenkesekvensid
    and a.relativ_posisjon between c.startposisjon and c.sluttposisjon
)

, har_eller_har_ikke as (
  select
    veglenkesekvensid
  , max(case when (rasteplass + dognhvileplass) > 0 then 1 else 0 end) as raste_eller_dognhvileplass
  from joined
  group by 1
)

select avg(b.raste_eller_dognhvileplass) as andel_vls
from joined as a
inner join har_eller_har_ikke as b
  on a.veglenkesekvensid = b.veglenkesekvensid
"""

#print(query)

client.query(query).to_dataframe()

,andel_vls
0,0.043461


Vi ser hvilken andel veglenkesekvenser som er tilknyttet raste eller døgnhvileplass

## Eksempel 7
Hvilken kommune og veitype hadde flest registrerte asfaltarbeid i 2022?

In [ ]:
query = f"""
-- NB: Logikken vil dobbeltelle i de tilfellene der samme asfaltarbeid traff flere kommuner.
with vegpunkter as (
  select * from saga-data.nvdb.nvdb_vegpunkter
  where vegkategori in ("E", "R")
)

, paved_road as (
  select id as asfalt_id
  , ls.veglenkesekvensid
  , ls.startposisjon
  , ls.sluttposisjon
  from `saga-nvdb-prod-vlmh.standardized.vegobjekter_vegdekke`, unnest(lokasjon.stedfestinger) ls
  where extract(year from egenskaper.dekkeleggingsdato) = 2022
  and metadata.sluttdato is null
)

, joined as (
  select a.*, b.asfalt_id
  from vegpunkter as a
  left join paved_road as b
    on a.veglenkesekvensid = b.veglenkesekvensid
    and a.relativ_posisjon between b.startposisjon and b.sluttposisjon
)

, koble_paa_kommunenavn as (
  select
    a.*, b.kommunenavn
  from joined as a
  left join saga-data.geografi.kommuner_2020_v2 as b
    on a.kommunenummer = b.kommunenummer
)

select
  kommunenavn,
  vegkategori,
  count(distinct asfalt_id) as antall_asfalteringer
from koble_paa_kommunenavn
group by 1,2
order by 3 desc
"""

#print(query)

client.query(query).to_dataframe()

,kommunenavn,vegkategori,antall_asfalteringer
0,Kristiansand,E,110
1,Eidfjord,R,48
2,Tysvær,E,33
3,Karmøy,E,25
4,Oslo,E,22
...,...,...,...
284,Larvik,R,0
285,Horten,R,0
286,Moskenes,R,0
287,Sør-Fron,E,0


Vi ser antall asfalteringer sortert på høyest øverst.

## Eksempel 8
Jeg har fått en måling med vegsystemreferanse med bokstaver og meter.<br>
Kan jeg få et datasett om denne strekningen?

**NB:** vegsystemreferanse kortform er bare gyldige frem til det skjer en endring på for vegsystemreferansen - f.eks. kan metreringen oppdaters. Husk å koble på nærmeste run_yearmonth fra nvdb_vegpunkter_history

In [ ]:
query = f"""
with eksempeldata as (
  select
    "EV6 S9D1 m0-70" as kortform,
    date("2023-10-05") as measurement_date,
    42 as measurement
)

-- Vegsystemreferansen må brytes opp på denne måten for å kobles til NVDB Vegpunkter
, eksempeldata_brutt_opp as (
  select
    trim(regexp_replace(kortform, regexp_extract(kortform, r"m\d+-\d+"), "")) as vl_vegsystemref_rot
  , cast(replace(split(regexp_extract(kortform, r"m\d+-\d+"), "-")[safe_offset(0)], "m", "") as int64) as vl_vegsystemref_fra_meter
  , cast(split(regexp_extract(kortform, r"m\d+-\d+"), "-")[safe_offset(1)] as int64) as vl_vegsystemref_til_meter
  , measurement
  from eksempeldata
)

, vegpunkter as (
  select *
  from saga-data.nvdb.nvdb_vegpunkter_history
  where run_yearmonth = (
    select cast(cast(extract(year from measurement_date) as string) || format("%02d", extract(month from measurement_date)) as int64) from eksempeldata
  )
)

select
  b.*,
  a.measurement
from eksempeldata_brutt_opp as a
inner join vegpunkter as b
  on a.vl_vegsystemref_rot = b.vl_vegsystemref_rot
  and b.metrering between a.vl_vegsystemref_fra_meter and a.vl_vegsystemref_til_meter
"""

#print(query)

client.query(query).to_dataframe()

,punkt_id,punkt_geometri,punkt_geohash,grader_ost,grader_nord,veglenke_referanse,veglenkesekvensid,veglenkenummer,segmentnummer,vl_index,...,vl_har_svingefelt,vl_har_sykkelfelt,vl_har_kollektivfelt,vl_har_bomstasjonfelt,run_time,run_yearmonth,square_gridcell_id_small,ssb_gridcell_id_1000m,h3_index_res_8,measurement
0,443149-15-13-20-0-u4xd7tz07nrym2xm09tc,POINT(10.710436 59.489671),u4xd7tz07nrym2xm09tc,10.710436,59.489671,443149-15-13,443149,15,13,2,...,False,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,1724683002218427622,22570006602000,88622e6ec9fffff,42
1,443149-15-13-9-0-u4xd7txp25cb8x8gn959,POINT(10.710299 59.489485),u4xd7txp25cb8x8gn959,10.710299,59.489485,443149-15-13,443149,15,13,2,...,False,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,1724683002218427622,22570006602000,88622e6ec9fffff,42
2,443149-15-13-4-2-u4xd7twyyzjkmyumuss9,POINT(10.7102502742136 59.4894172288519),u4xd7twyyzjkmyumuss9,10.710250,59.489417,443149-15-13,443149,15,13,2,...,False,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,1724683002218427622,22570006602000,88622e6ec9fffff,42
3,443149-15-13-4-0-u4xd7twyys459mjc8erv,POINT(10.710235 59.489401),u4xd7twyys459mjc8erv,10.710235,59.489401,443149-15-13,443149,15,13,2,...,False,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,1724683002218427622,22570006602000,88622e6ec9fffff,42
4,443149-15-13-22-1-u4xd7tz0sn250r0sch8y,POINT(10.7104683034105 59.4897134045473),u4xd7tz0sn250r0sch8y,10.710468,59.489713,443149-15-13,443149,15,13,2,...,False,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,1724683002218427622,22570006602000,88622e6ec9fffff,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,443149-15-13-24-0-u4xd7tz0u7j5yt0b072w,POINT(10.710486 59.489739),u4xd7tz0u7j5yt0b072w,10.710486,59.489739,443149-15-13,443149,15,13,2,...,False,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,8826570624018952660,22570006602000,88622e6ec9fffff,42
89,443149-15-13-18-1-u4xd7tz06cdmzqt111tq,POINT(10.7104179759812 59.4896454659734),u4xd7tz06cdmzqt111tq,10.710418,59.489645,443149-15-13,443149,15,13,2,...,False,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,1724683002218427622,22570006602000,88622e6ec9fffff,42
90,443149-15-13-18-0-u4xd7tz068n52y6pd2x7,POINT(10.710412 59.489637),u4xd7tz068n52y6pd2x7,10.710412,59.489637,443149-15-13,443149,15,13,2,...,False,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,1724683002218427622,22570006602000,88622e6ec9fffff,42
91,443149-15-13-6-0-u4xd7twzp4qnmxjx5sr5,POINT(10.710262 59.489435),u4xd7twzp4qnmxjx5sr5,10.710262,59.489435,443149-15-13,443149,15,13,2,...,False,False,False,False,2023-10-06 12:29:25.046604+00:00,202310,1724683002218427622,22570006602000,88622e6ec9fffff,42


Vipps, et datasett med mye mer info.



## Eksempel 9
I NVDB Vegpunker finnes tilknytning mellom unike vegpunkter og ruter i det minste kvadratiske rutenettet.<br>
Hvordan kan jeg gå fra små ruter til større ruter i de kvadratiske rutenettene?<br>
Spørringen under kan left joines mot NVDB Vegpunkter (square_gridcell_id_small).

In [ ]:
query = """
select
  s.square_gridcell_id_small,
  m.square_gridcell_id_medium,
  l.square_gridcell_id_large,
  s.geometri as squaregrid_geo_small,
  m.geometri as squaregrid_geo_medium,
  l.geometri as squaregrid_geo_large
from
  saga-data.geografi.norway_square_polygons_small s
left join saga-data.geografi.norway_square_polygons_medium m
  on s.x_index_medium = m.x_index_medium
  and s.y_index_medium = m.y_index_medium
left join saga-data.geografi.norway_square_polygons_large l
  on m.x_index_large = l.x_index_large
  and m.y_index_large = l.y_index_large
limit 20
;
"""

client.query(query).to_dataframe()

,square_gridcell_id_small,square_gridcell_id_medium,square_gridcell_id_large,squaregrid_geo_small,squaregrid_geo_medium,squaregrid_geo_large
0,1925793301446311868,5263060437513505542,-1660893193431286525,"POLYGON((18.017578125 69.1156110649945, 18.017...","POLYGON((18.12744140625 69.107776773315, 18.12...","POLYGON((18.78662109375 68.4718640372695, 18.7..."
1,-2877076736186226906,-8040690705973497860,5159646999497397617,"POLYGON((31.66259765625 69.1156110649945, 31.6...","POLYGON((31.75048828125 69.107776773315, 31.75...","POLYGON((31.97021484375 68.4718640372695, 31.9..."
2,-7693395584244387079,1224298317140387444,2605798031328101630,"POLYGON((27.79541015625 69.1156110649945, 27.7...","POLYGON((28.01513671875 69.107776773315, 28.01...","POLYGON((29.77294921875 68.4718640372695, 29.7..."
3,-1473120854722803697,1286409629780712777,2605798031328101630,"POLYGON((29.1357421875 69.1156110649945, 29.13...","POLYGON((29.33349609375 69.107776773315, 29.33...","POLYGON((29.77294921875 68.4718640372695, 29.7..."
4,2447355533177041818,-8478624301483686184,6397499869476659823,"POLYGON((16.3037109375 69.1156110649945, 16.30...","POLYGON((16.36962890625 69.107776773315, 16.36...","POLYGON((16.58935546875 68.4718640372695, 16.5..."
5,-7240107498459674451,5016410410774519245,-2882885412966951524,"POLYGON((19.4677734375 69.1156110649945, 19.46...","POLYGON((19.66552734375 69.107776773315, 19.66...","POLYGON((20.98388671875 68.4718640372695, 20.9..."
6,-3361127644284394657,5483952677618815887,-1907643926453264373,"POLYGON((13.4033203125 69.1156110649945, 13.40...","POLYGON((13.51318359375 69.107776773315, 13.51...","POLYGON((14.39208984375 68.4718640372695, 14.3..."
7,-5049755058366442855,6730753846802321741,7112209013372050791,"POLYGON((26.2353515625 69.1156110649945, 26.23...","POLYGON((26.25732421875 69.107776773315, 26.25...","POLYGON((27.57568359375 68.4718640372695, 27.5..."
8,5795042317016943063,-709119208164273138,7112209013372050791,"POLYGON((25.64208984375 69.1156110649945, 25.6...","POLYGON((25.81787109375 69.107776773315, 25.81...","POLYGON((27.57568359375 68.4718640372695, 27.5..."
9,-661002289731445521,-3325475211895801667,-2882885412966951524,"POLYGON((19.84130859375 69.1156110649945, 19.8...","POLYGON((19.88525390625 69.107776773315, 19.88...","POLYGON((20.98388671875 68.4718640372695, 20.9..."


## Eksempel 10
Jeg har en punkt_id i NVDB Vegpunkter<br>
Hvordan jeg kan gå fra punktets rute (i det minste kvadratiske rutenettet) til en større kvadratisk rute?<br>
I eksempelet er punkt_id'en gjeldende i run_yearmonth 202310 i NVDB Vegpunkter.

In [ ]:
query = """
with punkt as (
  select punkt_id, punkt_geometri, run_yearmonth
  from saga-data.nvdb.nvdb_vegpunkter_history
  where run_yearmonth = 202310 
  and punkt_id = "625212-24-14-12-5-u4xsw6vq9sk77h2mcgjc"
)
, grid_mapping as (
select
  a.punkt_id,
  b.punkt_geometri,
  a.* except (punkt_id)
from
  saga-data-stm.geografi.norway_square_polygons_to_nvdb_vegpunkter_mapping_history a
inner join punkt b
  on a.punkt_id = b.punkt_id
  and a.run_yearmonth = b.run_yearmonth
)
select
  a.*,
  b.geometri
from grid_mapping a
inner join saga-data.geografi.norway_square_polygons_small b
  on a.square_gridcell_id_small = b.square_gridcell_id_small
/*
inner join saga-data.geografi.norway_square_polygons_medium c
  on a.square_gridcell_id_medium = c.square_gridcell_id_medium
inner join saga-data.geografi.norway_square_polygons_large d
  on a.square_gridcell_id_large = d.square_gridcell_id_large
*/
;
"""

client.query(query).to_dataframe()

,punkt_id,punkt_geometri,square_gridcell_id_large,square_gridcell_id_medium,square_gridcell_id_small,run_yearmonth,geometri
0,625212-24-14-12-5-u4xsw6vq9sk77h2mcgjc,POINT(10.8288131798027 59.8697609919438),-3785447814842115875,7248919476060668661,-2209082758545707476,202310,"POLYGON((10.810546875 59.8779118748312, 10.810..."


## Eksempel 11
Heksagonale rutenett er hierarkiske, dvs. at enhver rute har en forelder-rute.<br>
Obs! Om du skal matche et unikt punkt mot ruter av forskjellige størrelser, bør du følge eksempelet under (Eksempel 12). Dette er fordi et punkt ikke nødvendig er innenfor forelder-rute (Y) til rute X, selv om hierarkiet for heksagonale ruter tilsier at rute X har forelder Y.<br><br>
Følgende SQL kan brukes som utgangspunkt for å visualisere ruter av forskjellige størrelser.

In [ ]:
query = """
with hexagons as (
  select
    r8.h3_index_res_8,
    r7.h3_index_res_7,
    r6.h3_index_res_6,
    r5.h3_index_res_5,
    r8.geometri as hexgrid_geo_r8,
    r7.geometri as hexgrid_geo_r7,
    r6.geometri as hexgrid_geo_r6,
    r5.geometri as hexgrid_geo_r5
  from
    saga-data.geografi.norway_hexagon_polygons_resolution_8 r8
  left join saga-data.geografi.norway_hexagon_polygons_resolution_7 r7
    on r8.parent_h3_index = r7.h3_index_res_7
  left join saga-data.geografi.norway_hexagon_polygons_resolution_6 r6
    on r7.parent_h3_index = r6.h3_index_res_6
  left join saga-data.geografi.norway_hexagon_polygons_resolution_5 r5
    on r6.parent_h3_index = r5.h3_index_res_5
)
select *
from hexagons
/*
-- Enkelte ruter har ikke en forelder.
-- Disse ligger likevel utenfor Norge, så det er ikke et problem.
where (h3_index_res_7 is null
  or h3_index_res_6 is null
  or h3_index_res_5 is null)
*/
;
"""

client.query(query).to_dataframe()

,h3_index_res_8,h3_index_res_7,h3_index_res_6,h3_index_res_5,hexgrid_geo_r8,hexgrid_geo_r7,hexgrid_geo_r6,hexgrid_geo_r5
0,8862d31211fffff,8762d3121ffffff,None,None,"POLYGON((3.93665963202251 60.9416969044522, 3....","POLYGON((3.94302122044578 60.9393103879786, 3....",None,None
1,8862d31213fffff,8762d3121ffffff,None,None,"POLYGON((3.93584696293133 60.9492951866348, 3....","POLYGON((3.94302122044578 60.9393103879786, 3....",None,None
2,8862d31219fffff,8762d3121ffffff,None,None,"POLYGON((3.94302122044578 60.9393103879786, 3....","POLYGON((3.94302122044578 60.9393103879786, 3....",None,None
3,8862d3121bfffff,8762d3121ffffff,None,None,"POLYGON((3.94220884573015 60.9469081227498, 3....","POLYGON((3.94302122044578 60.9393103879786, 3....",None,None
4,8862d3124bfffff,None,None,None,"POLYGON((3.95181741415406 60.9141356171176, 3....",None,None,None
5,8862d3125bfffff,8762d3125ffffff,None,None,"POLYGON((3.95574173098766 60.9345382379978, 3....","POLYGON((3.95655339234122 60.9269421000198, 3....",None,None
6,8862d31251fffff,8762d3125ffffff,None,None,"POLYGON((3.9501938759543 60.9293274805382, 3.9...","POLYGON((3.95655339234122 60.9269421000198, 3....",None,None
7,8862d3125dfffff,8762d3125ffffff,None,None,"POLYGON((3.95100570731275 60.9217312975575, 3....","POLYGON((3.95655339234122 60.9269421000198, 3....",None,None
8,8862d31257fffff,8762d3125ffffff,None,None,"POLYGON((3.94383347077941 60.931713155297, 3.9...","POLYGON((3.95655339234122 60.9269421000198, 3....",None,None
9,8862d31259fffff,8762d3125ffffff,None,None,"POLYGON((3.95655339234122 60.9269421000198, 3....","POLYGON((3.95655339234122 60.9269421000198, 3....",None,None


## Eksempel 12
Jeg har en punkt_id i NVDB Vegpunkter.<br>
Hvordan jeg kan gå fra punktets rute (i det minste heksagonale rutenettet) til en større heksagonal rute?<br>
I eksempelet er punkt_id'en gjeldende i run_yearmonth 202310 i NVDB Vegpunkter.

In [ ]:
query = """
with punkt as (
  select punkt_id, punkt_geometri, run_yearmonth
  from saga-data.nvdb.nvdb_vegpunkter_history
  where run_yearmonth = 202310 
  and punkt_id = "625212-24-14-12-5-u4xsw6vq9sk77h2mcgjc"

  
)
, grid_mapping as (
select
  a.punkt_id,
  b.punkt_geometri,
  a.* except (punkt_id)
from
  saga-data-stm.geografi.norway_hexagon_polygons_to_nvdb_vegpunkter_mapping_history a
inner join punkt b
  on a.punkt_id = b.punkt_id
  and a.run_yearmonth = b.run_yearmonth
)
select
  a.*,
  b.geometri
from grid_mapping a
inner join saga-data.geografi.norway_hexagon_polygons_resolution_8 b
  on a.h3_index_res_8 = b.h3_index_res_8
/*
inner join saga-data.geografi.norway_hexagon_polygons_resolution_7 c
  on a.h3_index_res_7 = c.h3_index_res_7
inner join saga-data.geografi.norway_hexagon_polygons_resolution_6 d
  on a.h3_index_res_6 = d.h3_index_res_6
inner join saga-data.geografi.norway_hexagon_polygons_resolution_5 e
  on a.h3_index_res_5 = e.h3_index_res_5
*/
;
"""

client.query(query).to_dataframe()

,punkt_id,punkt_geometri,h3_index_res_5,h3_index_res_6,h3_index_res_7,h3_index_res_8,run_yearmonth,geometri
0,625212-24-14-12-5-u4xsw6vq9sk77h2mcgjc,POINT(10.8288131798027 59.8697609919438),85622e37fffffff,86622e34fffffff,87622e34affffff,88622e34abfffff,202310,"POLYGON((10.8251146100539 59.8729141812931, 10..."
